In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import math
from collections import defaultdict
import csv
import pandas as pd
import numpy as np
from flask import jsonify
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.externals import joblib
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
data = pd.read_csv('house_prediction/data/output copy.csv')

# Separate out the x_data and y_data.
data_X = data.loc[:, [col for col in data.columns if col in ["latitude","longitude","city","state","postal_code","#bedrooms","#bathrooms","sqft"]]]
data_y = data.loc[:, "price"]
#print data_X

In [3]:
""" about label encoder
#https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
# Encoding the variable
fit = df.apply(lambda x: d[x.name].fit_transform(x))
# Inverse the encoded
fit.apply(lambda x: d[x.name].inverse_transform(x))
# Using the dictionary to label future data
df.apply(lambda x: d[x.name].transform(x))
"""
encode_list = ['city','state']
d = defaultdict(LabelEncoder)
data_X_encoded = data_X.apply(lambda x: d[x.name].fit_transform(x) if x.name in encode_list else x)
#print data_X_encoded

In [4]:
print d['city']
np.save('city.npy', d['city'].classes_)
np.save('state.npy', d['state'].classes_)
le_city = LabelEncoder()
le_city.classes_ = np.load('city.npy')
le_state = LabelEncoder()
le_state.classes_ = np.load('state.npy')
print le_city.transform(['Atlanta','Brookhaven','College Park','Dunwoody','Marietta','Sandy Springs','Smyrna'])


LabelEncoder()
[0 1 2 3 4 5 6]


In [6]:
#input format: latitude	longitude	city	state	postal_code
#  #bedrooms	#bathrooms	sqft	d1	d2	d3	d4	d5	d6	d7	d8	d9	d10	d11	d12	d13	d14	d15	d16	d17	d18	d19	d20
x_test = [33.913258,-84.451048,le_city.transform(['Atlanta'])[0],
          le_state.transform(['GA'])[0],30339,5,7,8407]


In [7]:
from sklearn.metrics import fbeta_score, make_scorer
def my_custom_loss_func(ground_truth, predictions):
    diff = -np.abs((ground_truth - predictions)/ground_truth).mean()
    return diff
my_scorer = make_scorer(my_custom_loss_func, greater_is_better=True)
cv_para = ShuffleSplit(n_splits=10, test_size=0.1, random_state=100)

In [24]:
nn_model = MLPRegressor(hidden_layer_sizes=(1, ), activation='relu',
             solver='adam', alpha=0.00001, batch_size='auto',
             learning_rate='constant', learning_rate_init=0.001,
             power_t=0.5, max_iter=10000, shuffle=True,
             random_state=None, tol=0.001, verbose=False, 
             warm_start=False, momentum=0.9, nesterovs_momentum=True, 
             early_stopping=False, validation_fraction=0.1,
             beta_1=0.9, beta_2=0.999, epsilon=1e-08)


print 'cross validation score of rf_model: '+str(cross_val_score(nn_model, data_X_encoded, data_y,n_jobs=3,verbose=5,cv=cv_para, scoring=my_scorer).mean())
   


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] .......................... , score=-0.666815486827, total=   7.0s
[CV]  ................................................................
[CV] .......................... , score=-0.623588034448, total=   7.2s
[CV]  ................................................................
[CV] .......................... , score=-0.617937194273, total=   7.5s
[CV]  ................................................................
[CV] .......................... , score=-0.614037661707, total=   8.1s
[CV]  ................................................................
[CV] .......................... , score=-0.660729768062, total=   9.6s
[CV]  ................................................................
[CV] .......................... , score=-0.628211864116, total= 1.2min
[CV]  

/Users/wqshen/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV] .......................... , score=-0.997970832152, total= 3.1min
[CV]  ................................................................


/Users/wqshen/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV] .......................... , score=-0.998051736409, total= 3.3min


[Parallel(n_jobs=3)]: Done   8 out of  10 | elapsed:  3.7min remaining:   55.8s


[CV] .......................... , score=-0.640464439604, total=  51.5s


/Users/wqshen/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV] .......................... , score=-0.998034374631, total= 2.8min
cross validation score of rf_model: -0.7445841392228868


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:  4.2min finished


In [9]:
from sklearn.svm import SVC
svc_model = SVC()
print 'cross validation score of rf_model: '+str(cross_val_score(svc_model, data_X_encoded, data_y,n_jobs=3,verbose=5,cv=cv_para, scoring=my_scorer).mean())
   


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] .......................... , score=-0.461521046711, total= 1.8min
[CV]  ................................................................
[CV] .......................... , score=-0.485031430779, total= 1.8min
[CV]  ................................................................
[CV] .......................... , score=-0.478265010351, total= 1.8min
[CV]  ................................................................
[CV] .......................... , score=-0.457180777178, total= 1.9min
[CV]  ................................................................
[CV] .......................... , score=-0.450722283666, total= 1.9min
[CV]  ................................................................
[CV] ........................... , score=-0.47736119045, total= 1.9min
[CV]  

[Parallel(n_jobs=3)]: Done   8 out of  10 | elapsed:  5.6min remaining:  1.4min


[CV] .......................... , score=-0.460421576576, total= 1.9min
[CV] .......................... , score=-0.477096477882, total= 1.2min
cross validation score of rf_model: -0.46858980264105565


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:  6.8min finished


In [8]:
from sklearn.preprocessing import StandardScaler, normalize
scaler = StandardScaler()
scaler.fit(data_X_encoded)
data_X_encoded_p = scaler.transform(data_X_encoded)
print data_X_encoded_p

[[ 0.17792227  0.54697493 -0.39687684 ... -1.53343023 -1.23569297
  -0.97767581]
 [ 1.33052822 -0.77654031 -0.39687684 ...  1.72579362  4.00495537
   6.37645612]
 [-0.3522976   0.6008744  -0.39687684 ...  0.09618169  0.07446912
   1.80089736]
 ...
 [ 0.96056049 -1.64351593  3.1885446  ... -0.71862427 -1.23569297
  -0.93491358]
 [ 1.07629065 -1.28557789  3.1885446  ...  0.09618169 -0.79897227
  -0.77261148]
 [-0.17704909 -0.33560701 -0.39687684 ...  0.91098766  0.51118981
   0.92718722]]


In [12]:
parameters = {'C':[0.001,0.1,10]}
clf = GridSearchCV(svc_model, parameters,scoring=my_scorer,cv=cv_para,verbose=30,n_jobs=3)
clf.fit(data_X_encoded_p,data_y)
print ("rf best para:",clf.best_params_)
print ("rf best score:",clf.best_score_)
print ("results",clf.cv_results_)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
[CV] C=0.001 .........................................................
[CV] C=0.001 .........................................................
[CV] C=0.001 .........................................................
[CV] ................... C=0.001, score=-0.548685773639, total= 1.8min
[CV] C=0.001 .........................................................


[Parallel(n_jobs=3)]: Done   1 tasks      | elapsed: 13.5min


[CV] ................... C=0.001, score=-0.552738766195, total= 1.8min
[CV] C=0.001 .........................................................


[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed: 13.7min


[CV] ................... C=0.001, score=-0.555131516748, total= 1.8min
[CV] C=0.001 .........................................................


...

[CV] ...................... C=10, score=-0.304298274157, total= 1.8min
[CV] C=10 ............................................................
[CV] ...................... C=10, score=-0.300108217563, total= 2.0min
[CV] C=10 ............................................................


[Parallel(n_jobs=3)]: Done  27 out of  30 | elapsed: 119.2min remaining: 13.2min


[CV] ...................... C=10, score=-0.309919726508, total= 1.6min
[CV] ...................... C=10, score=-0.287017812432, total= 1.7min
[CV] ...................... C=10, score=-0.305581637343, total= 1.7min


[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed: 131.2min finished


('rf best para:', {'C': 10})
('rf best score:', -0.30509534500757457)
('results', {'rank_test_score': array([3, 2, 1], dtype=int32), 'split6_test_score': array([-0.55097746, -0.49024989, -0.30010822]), 'split7_train_score': array([-0.54306008, -0.4259885 , -0.27006753]), 'split0_train_score': array([-0.54092719, -0.47195885, -0.26373434]), 'split2_test_score': array([-0.55273877, -0.49690995, -0.31976264]), 'mean_fit_time': array([18.44693127, 22.80918329, 27.83457992]), 'split3_train_score': array([-0.54170212, -0.49190211, -0.26397375]), 'param_C': masked_array(data=[0.001, 0.1, 10],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'split6_train_score': array([-0.54138875, -0.48103573, -0.26822729]), 'split9_test_score': array([-0.53216879, -0.42304827, -0.30558164]), 'std_test_score': array([0.00831368, 0.03244336, 0.01465758]), 'params': [{'C': 0.001}, {'C': 0.1}, {'C': 10}], 'std_fit_time': array([0.60951674, 0.9254287 , 1.29243519]), 'std

In [23]:
ridge1_model = Ridge(alpha=100, fit_intercept=True, normalize=False, copy_X=True, 
      max_iter=None, tol=0.001, solver='auto', random_state=None)
print 'cross validation score of rf_model: '+str(cross_val_score(ridge1_model, data_X_encoded_p, data_y,n_jobs=3,verbose=5,cv=cv_para, scoring=my_scorer).mean())
   


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] .......................... , score=-0.490700514525, total=   0.0s
[CV]  ................................................................
[CV] .......................... , score=-0.434454080605, total=   0.0s
[CV] .......................... , score=-0.485308229445, total=   0.0s
[CV] .......................... , score=-0.458384502219, total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] .......................... , score=-0.497703194798, total=   0.0s
[CV] .......................... , score=-0.477885625895, total=   0.0s
[CV]  ................................................................
[CV] ........................... , score=-0.47646771997, total=   0.0s
[CV]  

[Parallel(n_jobs=3)]: Done   5 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished


In [16]:
parameters = {'alpha':[0.0001,0.001,0.01,0.1,1,10,100]}
clfridge1 = GridSearchCV(ridge1_model, parameters,scoring=my_scorer,cv=cv_para,verbose=30,n_jobs=3)
clfridge1.fit(data_X_encoded_p,data_y)
print ("rf best para:",clfridge1.best_params_)
print ("rf best score:",clfridge1.best_score_)
print ("results",clfridge1.cv_results_)

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] .............. alpha=0.0001, score=-0.489634001408, total=   0.1s
[CV] .............. alpha=0.0001, score=-0.432766017941, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] .............. alpha=0.0001, score=-0.483605198348, total=   0.1s
[CV] .............. alpha=0.0001, score=-0.456300361862, total=   0.0s
[CV] alpha=0.0001 ....................................................
[CV] alpha=0.0001 ....................................................
[CV] .............. alpha=0.0001, score=-0.494684896224, total=   0.0s
[CV] alpha=0.0001 ....................................................
[CV] .............. alpha=0.0001, score=-0.475047196042, total=   0.0s
[CV] ...........

[Parallel(n_jobs=3)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Batch computation too fast (0.1295s.) Setting batch_size=2.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Batch computation too fast (0.1039s.) Setting batch_size=6.
[Parallel(n_jobs=3)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  16 tasks      | elapsed:    0.3s


[CV] ............... alpha=0.001, score=-0.459818046424, total=   0.0s
[CV] ............... alpha=0.001, score=-0.459870459792, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] alpha=0.001 .....................................................
[CV] ................ alpha=0.01, score=-0.494685192287, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] alpha=0.01 ......................................................
[CV] ............... alpha=0.001, score=-0.472638964783, total=   0.0s
[CV] ................. alpha=0.1, score=-0.489635081469, total=   0.0s
[CV] ................ alpha=0.01, score=-0.475047482521, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] alpha=0.01 ......................................................
[CV] alpha=0.1 .......................................................
[CV] ................ alpha=0.01, score=-0.489634108444, total=   0.0s
[CV] .

[Parallel(n_jobs=3)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done  24 tasks      | elapsed:    0.5s
[Parallel(n_jobs=3)]: Batch computation too fast (0.1943s.) Setting batch_size=12.
[Parallel(n_jobs=3)]: Done  30 tasks      | elapsed:    0.5s
[Parallel(n_jobs=3)]: Done  36 tasks      | elapsed:    0.6s


[CV] ................... alpha=1, score=-0.456321049624, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] alpha=1 .........................................................
[CV] alpha=1 .........................................................
[CV] ................... alpha=1, score=-0.459890129843, total=   0.0s
[CV] ................. alpha=0.1, score=-0.472640931786, total=   0.0s
[CV] alpha=1 .........................................................
[CV] alpha=1 .........................................................
[CV] ................... alpha=1, score=-0.494714951165, total=   0.0s
[CV] ................... alpha=1, score=-0.489644808551, total=   0.0s
[CV] alpha=1 .........................................................
[CV] alpha=1 .........................................................
[CV] ................... alpha=1, score=-0.472658807603, total=   0.0s
[CV] .................... alpha=1, score=-0.43278190566, total=   0.0s
[CV] .

[Parallel(n_jobs=3)]: Done  70 out of  70 | elapsed:    0.8s finished


In [ ]:
cv_score = np.array([-0.23824588, -0.20844122, -0.23356121, -0.21602981, -0.2285855,  -0.22002714,-0.21376681, -0.22638707, -0.21034281, -0.21976963])
cv_score.mean()

In [ ]:
nn_model.fit(data_X_encoded, data_y)


In [ ]:
#store the model
joblib.dump(nn_model, 'nn_model_without_d.pkl')
#to use saved model to predict
rf = joblib.load('nn_model_without_d.pkl')
rf.predict([x_test])